In [58]:
import pandas as pd 
import nltk

In [18]:
data = pd.read_csv('train3.csv')
data.drop('Id',axis=1 , inplace=True)
data

,Comment,Topic
0,A few things. You might have negative- frequen...,Biology
1,Is it so hard to believe that there exist part...,Physics
2,There are bees,Biology
3,I'm a medication technician. And that's alot o...,Biology
4,Cesium is such a pretty metal.,Chemistry
...,...,...
8690,I make similar observations over the last week...,Biology
8691,You would know.,Biology
8692,Also use the correct number of sig figs,Chemistry
8693,"What about the ethical delimmas, groundbreaki...",Biology


In [19]:
data.shape 
data.dtypes

Comment    object
Topic      object
dtype: object

In [20]:
CommentsToBeTokenized = data['Comment'] 
CommentsToBeTokenized

0       A few things. You might have negative- frequen...
1       Is it so hard to believe that there exist part...
2                                          There are bees
3       I'm a medication technician. And that's alot o...
4                          Cesium is such a pretty metal.
                              ...                        
8690    I make similar observations over the last week...
8691                                      You would know.
8692              Also use the correct number of sig figs
8693    What about the ethical delimmas,  groundbreaki...
8694                            I would like to know too.
Name: Comment, Length: 8695, dtype: object

In [21]:
tokens = CommentsToBeTokenized[0].split()
tokens

['A',
 'few',
 'things.',
 'You',
 'might',
 'have',
 'negative-',
 'frequency',
 'dependent',
 'selection',
 'going',
 'on',
 'where',
 'the',
 'least',
 'common',
 'phenotype,',
 'reflected',
 'by',
 'genotype,',
 'is',
 'going',
 'to',
 'have',
 'an',
 'advantage',
 'in',
 'the',
 'environment.',
 'For',
 'instance,',
 'if',
 'a',
 'prey',
 'animal',
 'such',
 'as',
 'a',
 'vole',
 'were',
 'to',
 'have',
 'a',
 'light',
 'and',
 'a',
 'dark',
 'phenotype,',
 'a',
 'predator',
 'might',
 'recognize',
 'the',
 'more',
 'common',
 'phenotype',
 'as',
 'food.',
 'So',
 'if',
 'the',
 'light',
 'voles',
 'are',
 'more',
 'common,',
 'foxes',
 'may',
 'be',
 'keeping',
 'a',
 'closer',
 'eye',
 'out',
 'for',
 'light',
 'phenotypic',
 'voles,',
 'recognising',
 'them',
 'as',
 'good',
 'prey.',
 'This',
 'would',
 'reduce',
 'the',
 'light',
 'causing',
 'alleles',
 'due',
 'to',
 'increased',
 'predation',
 'and',
 'the',
 'dark',
 'genotypes',
 'would',
 'increase',
 'their',
 'proport

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
bow = cv.fit_transform(CommentsToBeTokenized)

In [23]:
bow

<8695x18177 sparse matrix of type '<class 'numpy.int64'>'
	with 190076 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_result=tfidf.fit_transform(CommentsToBeTokenized).toarray()

In [25]:
tfidf_result.shape

(8695, 18177)

In [26]:
vectorizer = TfidfVectorizer(max_features=6000)
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
NB= MultinomialNB()

In [35]:
xtrain, xtest, ytrain, ytest = train_test_split(tfidf_result, data['Topic'], test_size= 0.2)
NB.fit(xtrain, ytrain)

MultinomialNB()

In [36]:
ypred1 = NB.predict(xtrain)
ypred = NB.predict(xtest)

In [37]:
ypred.shape

(1739,)

In [38]:
ytrain=ytrain[0:1739]
ytest=ytest[0:1739]
ypred1 = ypred1[0:1739]
ypred = ypred[0:1739]

In [39]:
print("Training Results:\n")
print(classification_report(ytrain, ypred1))
print("\nTesting Results:\n")
print(classification_report(ytest, ypred))

Training Results:

              precision    recall  f1-score   support

     Biology       0.73      0.95      0.82       709
   Chemistry       0.86      0.77      0.81       616
     Physics       0.95      0.61      0.75       414

    accuracy                           0.80      1739
   macro avg       0.85      0.78      0.79      1739
weighted avg       0.83      0.80      0.80      1739


Testing Results:

              precision    recall  f1-score   support

     Biology       0.64      0.89      0.74       730
   Chemistry       0.74      0.63      0.68       588
     Physics       0.88      0.48      0.62       421

    accuracy                           0.70      1739
   macro avg       0.75      0.66      0.68      1739
weighted avg       0.73      0.70      0.69      1739



In [ ]:
df = data.groupby('Topic')

In [ ]:
df.count()

,Comment
Topic,
Biology,3591
Chemistry,2920
Physics,2184


In [40]:
import sys # allows for command line arguments to be used as arguments for script.

# the 0th argument (shoud be in ""s) is the question
question = sys.argv[0] 
# the 1th argument ("Physics", "Chemistry", or "Biology") is the correct label to be checked with 
right_Label = sys.argv[1] 

In [56]:
def online_test(question, right_Label):
    # import the data
    dataList = {'Comment': question, 'Topic': right_Label}
    data.append(dataList, ignore_index=True)
    print (data) 
    CommentsToBeTokenized = data['Comment'] 
    print(data.iloc[-1,:])


    # vectorize the question
    vectorizer = TfidfVectorizer(max_features=6000)
    tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
    features = vectorizer.get_feature_names_out()
    tfidf_result = pd.DataFrame(tfidf_result, columns=features)
    
    # predict the label
    prediction = NB.predict(tfidf_result)
    print(data.iloc[:-1])
    print(prediction)
    # check if the prediction is correct
    if prediction[-1] == right_Label:
        return "Correct"
    else:
        return "Incorrect"



In [57]:
online_test("In another example, STP will be associated with 1 atm and 273 K. For part (b), the ideal gas equation (PV = nRT) and the solution (~22.4 L/mol) will be given to the machine. Based on this question and similar questions which may ask for other" , "Chemistry")

C:\Users\serap\AppData\Local\Temp\ipykernel_45420\3326796510.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data.append(dataList, ignore_index=True)


                                                Comment      Topic
0     A few things. You might have negative- frequen...    Biology
1     Is it so hard to believe that there exist part...    Physics
2                                        There are bees    Biology
3     I'm a medication technician. And that's alot o...    Biology
4                        Cesium is such a pretty metal.  Chemistry
...                                                 ...        ...
9150  (a) If an electric car is capable of going 225...  Chemistry
9151  The density of air at ordinary atmospheric pre...  Chemistry
9152  The concentration of carbon monoxide in an urb...  Chemistry
9153  Gold can be hammered into extremely thin sheet...  Chemistry
9154  A copper refinery produces a copper ingot weig...  Chemistry

[9155 rows x 2 columns]
Comment    A copper refinery produces a copper ingot weig...
Topic                                              Chemistry
Name: 9154, dtype: object
                       

'Correct'

In [ ]:
def online_test(question, right_Label):
    # import the data
    dataList = {'Comment': question, 'Topic': right_Label}
    data.append(dataList, ignore_index=True)
    CommentsToBeTokenized = data['Comment'] 
    print(data.iloc[-1])

    # tokenize the question
    tokens = question.split()

    # vectorize the question
    vectorizer = TfidfVectorizer(max_features=6000)
    tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
    features = vectorizer.get_feature_names_out()
    tfidf_result = pd.DataFrame(tfidf_result, columns=features)
    
    # predict the label
    prediction = NB.predict(tfidf_result)
    results = {'Comment': question, 'Topic': right_Label, 'Guess' = prediction[-1]}
    print(data)
    print(prediction)
    print(results) 
    # check if the prediction is correct
    if prediction[-1] == right_Label:
        return "Correct"
    else:
        return "Incorrect"

: 

In [61]:
import online_vectorizers
vectorizer = OnlineTfidfVectorizer(max_features=6000)
tfidf_result = vectorizer.fit(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)
#OnlineTfidfVectorizer object has no attribute '_tfidf'
#Why is this happening 

AttributeError: 'OnlineTfidfVectorizer' object has no attribute '_tfidf'

In [ ]:
def online_test(question, right_Label):
    
    # predict the label
    prediction = pd.DataFrame([question], columns= ['Comments'])
    prediction['Topic'] = right_Label
    print (prediction)
    tfidf_result=tfidf.fit_transform(prediction['Comments']).toarray()
    print (tfidf_result.shape) 
    vectorizer = TfidfVectorizer(max_features=len(question.split(" ")))
    tfidf_result=tfidf.fit_transform(prediction).toarray()
    print ( tfidf_result )
    print (len(question.split (" ")))

: 

: 